In [48]:
import pandas as pd
import numpy as np
import json

In [49]:
df = pd.read_csv("processed_data/merged_data.csv")
df.head()

,subject_id,hadm_id,subgroup_x,rounded_age,Combined_Blood_Pressure,charttime,Final_Temperature_F,heart_rate,RespiratoryRate_combined,SpO2,...,coagulopathy,obesity,weight_loss,fluid_electrolyte,blood_loss_anemia,deficiency_anemias,alcohol_abuse,drug_abuse,psychoses,depression
0,3,145834,6,77,99.0,2101-10-21 01:00:00,98.200043,141.0,0.0,97.0,...,0,0,1,0,0,0,0,0,0,0
1,3,145834,6,77,124.0,2101-10-21 02:00:00,97.160004,117.0,18.0,100.0,...,0,0,1,0,0,0,0,0,0,0
2,3,145834,6,77,129.0,2101-10-21 03:00:00,97.700000,114.0,18.0,100.0,...,0,0,1,0,0,0,0,0,0,0
3,3,145834,6,77,122.0,2101-10-21 04:00:00,97.879997,116.0,0.0,98.0,...,0,0,1,0,0,0,0,0,0,0
4,3,145834,6,77,114.0,2101-10-21 05:00:00,97.879997,108.0,0.0,100.0,...,0,0,1,0,0,0,0,0,0,0


In [50]:
df2 = pd.read_csv("processed_data/subgroup_probabilities.csv", index_col=0)
df2['max_value'] = df2[['prob_sub1', 'prob_sub2', 'prob_sub3', 'prob_sub4', 'prob_sub5', 'prob_sub6', 'prob_sub7']].max(axis=1)*100
df2.head()

,hadm_id,admission_type_encoded,congestive_heart_failure,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,peripheral_vascular,hypertension,paralysis,other_neurological,...,age,subgroup,prob_sub1,prob_sub2,prob_sub3,prob_sub4,prob_sub5,prob_sub6,prob_sub7,max_value
subject_id,,,,,,,,,,,,,,,,,,,,,
453,187961,1,1,0,1,0,0,1,0,0,...,80,1,1.00,0.00,0.00,0.00,0.0,0.00,0.0,100.0
1113,128609,1,1,0,0,0,0,1,0,1,...,46,1,0.74,0.01,0.24,0.00,0.0,0.01,0.0,74.0
1113,128609,1,1,0,0,0,0,1,0,1,...,46,1,0.74,0.01,0.24,0.00,0.0,0.01,0.0,74.0
3868,107365,1,0,1,0,0,0,1,0,0,...,65,1,0.94,0.00,0.00,0.06,0.0,0.00,0.0,94.0
22202,101241,1,1,1,0,0,0,1,0,0,...,66,1,1.00,0.00,0.00,0.00,0.0,0.00,0.0,100.0


In [51]:
df = df.merge(df2[['hadm_id', 'max_value']], how='left', on='hadm_id')
df.head()

,subject_id,hadm_id,subgroup_x,rounded_age,Combined_Blood_Pressure,charttime,Final_Temperature_F,heart_rate,RespiratoryRate_combined,SpO2,...,obesity,weight_loss,fluid_electrolyte,blood_loss_anemia,deficiency_anemias,alcohol_abuse,drug_abuse,psychoses,depression,max_value
0,3,145834,6,77,99.0,2101-10-21 01:00:00,98.200043,141.0,0.0,97.0,...,0,1,0,0,0,0,0,0,0,NaN
1,3,145834,6,77,124.0,2101-10-21 02:00:00,97.160004,117.0,18.0,100.0,...,0,1,0,0,0,0,0,0,0,NaN
2,3,145834,6,77,129.0,2101-10-21 03:00:00,97.700000,114.0,18.0,100.0,...,0,1,0,0,0,0,0,0,0,NaN
3,3,145834,6,77,122.0,2101-10-21 04:00:00,97.879997,116.0,0.0,98.0,...,0,1,0,0,0,0,0,0,0,NaN
4,3,145834,6,77,114.0,2101-10-21 05:00:00,97.879997,108.0,0.0,100.0,...,0,1,0,0,0,0,0,0,0,NaN


In [52]:
def convert_int64(dataframe):
    for col in dataframe.columns:
        if dataframe[col].dtype == np.int64:
            dataframe[col] = dataframe[col].astype(int)
    return df
df = convert_int64(df)

In [53]:
def get_latest_vitals(dataframe):
    dataframe['charttime'] = pd.to_datetime(dataframe['charttime'])
    dataframe_sorted = dataframe.sort_values(by=['subject_id', 'charttime'])
    latest_vitals = dataframe_sorted.groupby('subject_id').last().reset_index()
    return latest_vitals

latest_vitals_df = get_latest_vitals(df)

latest_vitals_df.head()

,subject_id,hadm_id,subgroup_x,rounded_age,Combined_Blood_Pressure,charttime,Final_Temperature_F,heart_rate,RespiratoryRate_combined,SpO2,...,obesity,weight_loss,fluid_electrolyte,blood_loss_anemia,deficiency_anemias,alcohol_abuse,drug_abuse,psychoses,depression,max_value
0,3,145834,6,77,117.0,2101-10-26 15:00:00,97.100057,88.0,19.0,97.0,...,0,1,0,0,0,0,0,0,0,NaN
1,4,185777,3,48,113.0,2191-03-16 17:00:00,97.400002,85.0,32.0,98.0,...,0,1,1,0,0,0,0,0,0,NaN
2,6,107064,4,66,144.0,2175-06-03 03:00:00,96.699997,84.0,10.0,98.0,...,0,0,1,0,0,0,0,0,0,NaN
3,9,150750,6,42,117.0,2149-11-14 10:00:00,96.000079,86.0,14.0,100.0,...,0,0,1,0,0,0,0,0,0,NaN
4,12,112213,4,72,147.0,2104-08-15 14:30:00,97.800079,82.0,26.0,95.0,...,0,0,0,0,0,0,0,0,0,NaN


In [54]:
latest_vitals_df[latest_vitals_df['predictions'] == 2]

,subject_id,hadm_id,subgroup_x,rounded_age,Combined_Blood_Pressure,charttime,Final_Temperature_F,heart_rate,RespiratoryRate_combined,SpO2,...,obesity,weight_loss,fluid_electrolyte,blood_loss_anemia,deficiency_anemias,alcohol_abuse,drug_abuse,psychoses,depression,max_value
1,4,185777,3,48,113.0,2191-03-16 17:00:00,97.400002,85.0,32.0,98.0,...,0,1,1,0,0,0,0,0,0,NaN
311,439,137678,6,83,155.0,2104-11-02 20:00:00,98.800003,107.0,26.0,99.0,...,0,0,1,0,0,0,0,0,0,NaN
319,453,187961,1,80,107.0,2156-04-26 15:00:00,99.600079,95.0,19.0,95.0,...,0,0,0,0,0,0,0,0,0,100.0
422,619,167213,3,48,108.0,2105-12-26 16:00:00,98.200043,65.0,30.0,98.0,...,0,0,1,0,0,0,0,0,0,NaN
427,625,145523,4,60,109.0,2178-05-03 12:00:00,96.600021,91.0,13.0,94.0,...,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21107,32489,166861,3,44,93.0,2184-08-22 11:00:00,97.400002,82.0,24.0,100.0,...,0,0,1,1,0,1,0,0,0,NaN
21131,32529,148697,3,68,99.0,2140-11-03 14:00:00,98.200043,80.0,40.0,100.0,...,0,0,0,0,0,0,0,0,0,NaN
21180,32624,118491,3,45,106.0,2106-06-20 12:45:00,96.800003,88.0,24.0,93.0,...,0,0,1,0,0,1,0,0,1,NaN
21212,32675,170865,3,58,78.0,2120-06-20 19:00:00,97.000000,73.0,28.0,98.0,...,0,0,1,0,0,0,0,0,0,NaN


In [55]:
latest_vitals_df.columns

Index(['subject_id', 'hadm_id', 'subgroup_x', 'rounded_age',
       'Combined_Blood_Pressure', 'charttime', 'Final_Temperature_F',
       'heart_rate', 'RespiratoryRate_combined', 'SpO2', 'date', 'hadm_number',
       'hours_in_hospital', 'sofa', 'group', 'predictions',
       'congestive_heart_failure', 'cardiac_arrhythmias', 'valvular_disease',
       'pulmonary_circulation', 'peripheral_vascular', 'hypertension',
       'paralysis', 'other_neurological', 'chronic_pulmonary',
       'diabetes_uncomplicated', 'diabetes_complicated', 'hypothyroidism',
       'renal_failure', 'liver_disease', 'peptic_ulcer', 'aids', 'lymphoma',
       'metastatic_cancer', 'solid_tumor', 'rheumatoid_arthritis',
       'coagulopathy', 'obesity', 'weight_loss', 'fluid_electrolyte',
       'blood_loss_anemia', 'deficiency_anemias', 'alcohol_abuse',
       'drug_abuse', 'psychoses', 'depression', 'max_value'],
      dtype='object')

In [69]:
def json_converter(dataframe, patient_id):
    data = {
        "name": "John Doe",
    }
    row = dataframe[dataframe['subject_id']==patient_id]
    
    elix = row.iloc[:, -31:-1]
    one_hot_columns = elix.columns[elix.eq(1).any()].tolist()
    one_hot_columns = [' '.join(word.capitalize() for word in item.replace('_', ' ').split()) for item in one_hot_columns]

    def custom_serializer(obj):
        if isinstance(obj, np.integer):
            return int(obj)  # Convert NumPy integers to Python int
        elif isinstance(obj, pd.Timestamp):
            return obj.isoformat()  # Convert Timestamp to ISO 8601 string
        raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")
    
    data["subject_id"] = row["subject_id"].iloc[0]
    data["subgroup"] = row["subgroup_x"].iloc[0]
    data["latest_blood_pressure"] = row["Combined_Blood_Pressure"].iloc[0]
    data["latest_chart_time"] = row["charttime"].iloc[0]
    data["latest_temp_F"] = round(row["Final_Temperature_F"].iloc[0], 2)
    data["latest_heart_rate"] = row["heart_rate"].iloc[0]
    data["latest_respiratory_rate"] = row["RespiratoryRate_combined"].iloc[0]
    data["elixhauser"] = one_hot_columns
    data["age"] = row["rounded_age"].iloc[0]
    data["spo2"] = row["SpO2"].iloc[0]
    data["sofa"] = row["sofa"].iloc[0]
    temp1=row["max_value"].iloc[0]
    if temp1==100.0:
        temp1=99.0
    data["prob"] = temp1
    data["white_blood"] = row["WBC_combined"].iloc[0]*1000
    
    file_name = "processed_data/id" + str(patient_id) + ".json"

    with open(file_name, 'w') as json_file:
        json.dump(data, json_file, indent=4, default=custom_serializer)

In [70]:
patient_list = [26494,
 6702,
 11167,
 3868,
 8566,
 6659,
 17797,
 1113,
 12613,
 25699,
 17807,
 7354,
 11952,
 17795,
 22202,
 3011,
 6873,
 25111,
 16847,
 145,
 9467]
for x in patient_list:
    json_converter(final_df, x)

In [9]:
latest_vitals_df[latest_vitals_df['subject_id']==59]['charttime'].iloc[0]

Timestamp('2197-11-16 19:00:00')

In [59]:
df3 = pd.read_csv("processed_data/white_bloodcells_with_sofa.csv", index_col=0)
df3.head()

,hadm_id,charttime,WBC_combined,subject_id,sofa
0,145834,2101-10-20 19:26:00,11.3,3,7.0
1,145834,2101-10-20 19:59:00,19.1,3,7.0
2,145834,2101-10-21 03:00:00,24.4,3,7.0
3,145834,2101-10-21 19:06:00,18.8,3,7.0
4,145834,2101-10-22 04:00:00,15.1,3,7.0


In [60]:
white = get_latest_vitals(df3)
white.head()

,subject_id,hadm_id,charttime,WBC_combined,sofa
0,3,145834,2101-10-26 03:00:00,10.5,7.0
1,4,185777,2191-03-17 06:00:00,7.3,3.0
2,6,107064,2175-06-03 00:58:00,4.9,4.0
3,9,150750,2149-11-14 06:00:00,11.1,4.0
4,11,194540,2178-04-17 02:35:00,12.8,1.0


In [65]:
final_df = pd.merge(latest_vitals_df, white[['subject_id', 'WBC_combined']], on='subject_id', how='left')
final_df.head()

,subject_id,hadm_id,subgroup_x,rounded_age,Combined_Blood_Pressure,charttime,Final_Temperature_F,heart_rate,RespiratoryRate_combined,SpO2,...,weight_loss,fluid_electrolyte,blood_loss_anemia,deficiency_anemias,alcohol_abuse,drug_abuse,psychoses,depression,max_value,WBC_combined
0,3,145834,6,77,117.0,2101-10-26 15:00:00,97.100057,88.0,19.0,97.0,...,1,0,0,0,0,0,0,0,NaN,10.5
1,4,185777,3,48,113.0,2191-03-16 17:00:00,97.400002,85.0,32.0,98.0,...,1,1,0,0,0,0,0,0,NaN,7.3
2,6,107064,4,66,144.0,2175-06-03 03:00:00,96.699997,84.0,10.0,98.0,...,0,1,0,0,0,0,0,0,NaN,4.9
3,9,150750,6,42,117.0,2149-11-14 10:00:00,96.000079,86.0,14.0,100.0,...,0,1,0,0,0,0,0,0,NaN,11.1
4,12,112213,4,72,147.0,2104-08-15 14:30:00,97.800079,82.0,26.0,95.0,...,0,0,0,0,0,0,0,0,NaN,30.2
